# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
weather_df = pd.read_csv("weather.csv")
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,0,tazovskiy,100,RU,1580664988,95,67.47,78.70,-9.85,20.65
1,1,puerto ayora,20,EC,1580664988,70,-0.74,-90.35,84.20,14.99
2,2,brae,20,GB,1580664989,80,60.40,-1.35,35.60,5.82
3,3,hirara,0,JP,1580664989,57,24.80,125.28,63.91,7.58
4,4,rikitea,100,PF,1580664989,77,-23.12,-134.97,79.48,16.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
%matplotlib inline
gmaps.configure(api_key=g_key)
#locations = gmaps.datasets.load_dataset_as_df('acled_africa')
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]
#locations.head()
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations,weights=humidity)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
 # A max temperature lower than 80 degrees but higher than 70.
 # Wind speed less than 10 mph.
 # Zero cloudiness.
 # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal
best_weather=weather_df[(weather_df.Max_Temp >= 70) & (weather_df.Max_Temp < 80) &  (weather_df.Wind_Speed < 10) & (weather_df.Cloudiness < 10)]
best_weather = best_weather.reset_index()
del best_weather["index"]
best_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,26,bredasdorp,6,ZA,1580664845,83,-34.53,20.04,71.60,9.17
1,71,araouane,0,ML,1580665040,9,18.90,-3.53,76.44,7.40
2,78,avarua,0,CK,1580665041,78,-21.21,-159.78,78.80,4.70
3,133,mangrol,8,IN,1580665090,68,21.12,70.12,73.53,6.82
4,141,gao,0,ML,1580665091,8,16.64,1.64,78.73,8.72
5,187,arlit,0,NE,1580665097,10,18.74,7.39,77.18,9.48
6,192,emerald,0,AU,1580665098,78,-23.53,148.17,75.20,5.82
7,246,ponta do sol,0,PT,1580665105,72,32.67,-17.10,71.01,9.17
8,268,carnarvon,0,AU,1580665129,94,-24.87,113.63,73.40,4.70
9,376,salalah,0,OM,1580665183,60,17.02,54.09,75.20,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df= best_weather
hotels = []

for city in range(len(best_weather["City"])):
    lat = best_weather.loc[city]["Lat"]     
    lng = best_weather.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {"location": city_coords, 
              "types": "lodging",
              "radius": 5000,
              "key": g_key
             }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")
hotel_df["Hotel Name"]=hotels
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed,Hotel Name
0,26,bredasdorp,6,ZA,1580664845,83,-34.53,20.04,71.60,9.17,Bredasdorp Country Manor
1,71,araouane,0,ML,1580665040,9,18.90,-3.53,76.44,7.40,Nearest hotel not found
2,78,avarua,0,CK,1580665041,78,-21.21,-159.78,78.80,4.70,Paradise Inn
3,133,mangrol,8,IN,1580665090,68,21.12,70.12,73.53,6.82,Hotel The Royal Honours
4,141,gao,0,ML,1580665091,8,16.64,1.64,78.73,8.72,Nearest hotel not found
5,187,arlit,0,NE,1580665097,10,18.74,7.39,77.18,9.48,Hôtel Telwa Bungalow
6,192,emerald,0,AU,1580665098,78,-23.53,148.17,75.20,5.82,The Irish Village
7,246,ponta do sol,0,PT,1580665105,72,32.67,-17.10,71.01,9.17,Hotel do Campo
8,268,carnarvon,0,AU,1580665129,94,-24.87,113.63,73.40,4.70,Hospitality Carnarvon
9,376,salalah,0,OM,1580665183,60,17.02,54.09,75.20,3.36,HAMDAN PLAZA HOTEL


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_weather.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
info_box_content= [f"Hotel Name: {hotel}" for hotel in hotels]
#print(info_box_content)
markers = gmaps.marker_layer(locations, info_box_content)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))